In [1]:
import pickle
import os
import matplotlib.pyplot as plt
from imageio import imread
import math
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
import math
import logging
import downloader
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# TODO: remove global...
CM_TO_KM = 100000           # CM in a KM
MAX_MAP_SIZE = 8            # In KM


# search for specific attributes 
# player_id :   players acount id (eg. 'acount.*****')
# start_range : starting time (eg. 'YYYY-MM-DDTHH:MM:SS.SSSZ')
# end_range:    ending time 
# event_type:   type of event (eg. 'LogParachutepiLanding')
def search(json_object, player_id=None, start_range=None, end_range=None, event_type=None):
    events = []
    i = 0
    for entry in json_object: 
        if (is_event(entry, event_type) and
                in_range(entry['_D'], start_range, end_range) and
                has_player(entry, player_id)):
            
            events.append(entry)
    return events


def in_range(time, start, end):
    if start is None and end is None:
        return True
    elif start is None:
        return is_after(end, time)
    elif end is None:
        return is_after(time, start)
    else:
        return is_after(time, start) and is_after(end, time)


def is_after(time1, time2):
    t_index = time1.find('T')
    date1 = time1[:t_index].encode('ascii', 'ignore')
    date1 = date1.split('-')
    time1 = time1[t_index + 1:][:-1].encode('ascii', 'ignore')
    time1 = time1.split(':')

    t_index = time2.find('T')
    date2 = time2[:t_index].encode('ascii', 'ignore')
    date2 = date2.split('-')
    time2 = time2[t_index + 1:][:-1].encode('ascii', 'ignore')
    time2 = time2.split(':')

    equals = True

    for x in range(3):
        if int(date1[x]) > int(date2[x]):
                return True
    for x in range(2):
        if int(time1[x]) > int(time2[x]):
                return True

    if float(time1[2]) > float(time2[2]):
        return True

    if time1 != time2:
        return False

    return equals


def has_player(event, player_id):
    if player_id is None:
        return True

    for key in ['character', 'attacker', 'victim', 'assistant', 'reviver']:
        if key in event.keys():
            if event[key]['accountId'] == player_id:
                return True
        else:
            return False
    return False


def is_event(event, event_type):
    if event_type is None:
        return True
    else:
        return event_type == event['_T']


# Return the dict from the pickle file name
def load_pickle(pickle_file):
    with open(pickle_file, 'rb') as f:
        telemetry_data = pickle.load(f)
    f.close()
    return telemetry_data


# Get the location the plane starts in (not quite sure it's useful yet...)
def get_plane_start(telemetry):
    start_loc = None
    for log_entry in telemetry:
        if log_entry["_T"] == "LogMatchStart":
            # All players start at the exact same location, so we only need first
            start_loc = log_entry["characters"][0]['location']
    return start_loc


# Get the map name from telemetry
def get_map(telemetry):
    for log_entry in telemetry:
        if log_entry["_T"] == "LogMatchStart":
            return log_entry["mapName"]
    return None


# Get the result for each player in the match from the telemetry data
# Return the results as a list of dicts:
#       [{'name' : playerName, 'ranking': ranking},
#        ...]
def get_rankings(telemetry):
    results = []
    for log_entry in telemetry:
        if log_entry["_T"] == "LogMatchEnd":
            for character in log_entry['characters']:
                results.append({'name': character['name'],
                                'ranking': character['ranking']})
    return results


# Get the location of each player's landing from the telemetry data
# Return as a dict:
#           {playerName: [x, y],
#            ...}
def get_all_landings(telemetry):
    landings = {}
    for log_entry in telemetry:
        if log_entry["_T"] == "LogParachuteLanding":
            landing_loc = log_entry['character']['location']
            x = landing_loc['x']
            y = landing_loc['y']
            landings[log_entry['character']['name']] = [x, y]
    return landings


# Returns the first  and last locations someone jumped out of the plane
def get_flight_data(telemetry):
    first_coordinate = None # First player exit event from plane
    current_coordinate = None # Last player exist even from plane
    for log_entry in telemetry:
        if log_entry.get("_T") == "LogVehicleLeave" \
                and log_entry.get("vehicle").get("vehicleId") == "DummyTransportAircraft_C":
            current_coordinate = log_entry.get("character").get("location")
            if first_coordinate is None:
                first_coordinate = current_coordinate
    return first_coordinate, current_coordinate


# Returns the unit vector for the flight path given the first and last location someone exited plane
def get_flight_vector(first_drop, last_drop):
    flight_vector = np.array([last_drop['x'] - first_drop['x'], last_drop['y'] - first_drop['y']])
    vector_length = np.linalg.norm(flight_vector)
    flight_vector_norm = flight_vector / vector_length
    return flight_vector_norm


# Returns the angle between the two flight vectors u and v (squared)
# Note: u and v must be unit vectors already
def flight_diff(u, v):
    return np.arccos(np.clip(np.dot(u, v), -1, 1))**2


# Given a flight path (unit vectorized), get the direction that it is closest to
# from one of the eight recognized cardinal directions
def get_flight_category(flight_vector):
    # Set up unit vectors for the 8 directions to classify flight paths as
    between_mag = 1 / math.sqrt(2)
    nn = [0, 1]
    ne = [between_mag, between_mag]
    ee = [1, 0]
    se = [between_mag, -between_mag]
    ss = [0, -1]
    sw = [-between_mag, -between_mag]
    ww = [-1, 0]
    nw = [-between_mag, between_mag]

    # Put em in a DataFrame for processing
    directions = np.array([nn, ne, ee, se, ss, sw, ww, nw])
    dirs_df = pd.DataFrame(directions)

    # Combine the x, y component columns, add direction names, and get rid of the excess
    dirs_df['direction_vec'] = list(zip(dirs_df[0], dirs_df[1]))
    dirs_df['direction'] = np.array(['nn', 'ne', 'ee', 'se', 'ss', 'sw', 'ww', 'nw'])
    dirs_df.drop([0, 1], axis=1, inplace=True)

    # Calculate the angle between the flight_vector and each of the 8 directions
    # and store it under a new column, 'angle_from_path'
    dirs_df['angle_from_path'] = dirs_df['direction_vec'].apply(flight_diff, args=(flight_vector,))

    # And return the direction (string) where the angle between the flight_vector
    # and the direction is minimized (i.e. the closest direction to the flight_vector)
    return dirs_df['direction'].loc[dirs_df['angle_from_path'].idxmin()]


# Convert raw (x, y) coordinates to the category
# Maps are divided into a 20x20 grid of square_size x square_size blocks
# where each square can be represented by a letter for the x and y position of that
# square in the map, e.g. AA for the square containing (0, 0)
def get_loc_cat(x, y, map_dim):
    square_size = 0.05 * map_dim
    x_ = x // square_size
    y_ = y // square_size

    # add the number square along each access to 'A's ascii code and return the characters
    # as the category
    return chr(65 + int(x_)) + chr(65 + int(y_))


# Plot the drop locations of each player (in blue), with opacity in relation to their rank in that match
# (more opaque = lower rank), along with the location the first person left the plane (in green)
# and the last person to leave the plane (in red)
def display_drop_locations(telemetry, fig, fig_x, fig_y, fig_num, match_num):
    landings = get_all_landings(telemetry)
    rankings = get_rankings(telemetry)
    map_name = get_map(telemetry)

    # Set up plot scale
    if map_name == "Savage_Main":                        # 4km map
        x_max = MAX_MAP_SIZE * (1/2)
        y_max = x_max
        map_img = imread("savage.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 4.0, 0.0, 4.0])
    elif map_name == "Erangel_Main":                     # 8km map
        x_max = MAX_MAP_SIZE
        y_max = MAX_MAP_SIZE
        map_img = imread("erangel.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 8.0, 0.0, 8.0])
    elif map_name ==  "Desert_Main":                     # 8km map
        x_max = MAX_MAP_SIZE
        y_max = x_max
        map_img = imread("miramar.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 8.0, 0.0, 8.0])
    elif map_name == "DihorOtok_Main":                   # 6km map
        x_max = MAX_MAP_SIZE * (3/4)
        y_max = x_max
        map_img = imread("vikendi.png")
        plt.imshow(map_img, zorder=0, extent=[0.0, 6.0, 0.0, 6.0])

    first_launch, last_launch = get_flight_data(telemetry)

    if first_launch is not None:
        launch_x = [first_launch['x'], last_launch['x']]
        launch_y = [first_launch['y'], last_launch['y']]

        ax = fig.add_subplot(fig_x, fig_y, fig_num)

        # plot first and last jump locations
        ax.scatter(launch_x[0] / CM_TO_KM, launch_y[0] / CM_TO_KM, s=100,
                   color='green', edgecolors='black', zorder=1)
        ax.scatter(launch_x[1] / CM_TO_KM, launch_y[1] / CM_TO_KM, s=100,
                   color='red', edgecolors='black', zorder=1)

        # plot line between them
        ax.plot([x_ / CM_TO_KM for x_ in launch_x],
                [y_ / CM_TO_KM for y_ in launch_y], 'grey', linestyle='--', marker='', zorder=1)

        # plot each player according to their ranking
        for ranking in rankings:
            landing_loc = landings[ranking['name']]
           # print("Player {} landing at position\t ({}, {}) and ended up rank : {}".format(ranking['name'],
            #                                                                           landing_loc[0],
             #                                                                          landing_loc[1],
             #                                                                          ranking['ranking']))
            if ranking['ranking'] == 1:
                ax.scatter(landing_loc[0] / CM_TO_KM, landing_loc[1] / CM_TO_KM,
                           color='yellow', edgecolors='black', zorder=1)
            else:
                ax.scatter(landing_loc[0] / CM_TO_KM, landing_loc[1] / CM_TO_KM,
                           color='blue', alpha=1/ranking['ranking'], zorder=1)
        plt.ylim(0, y_max)
        plt.xlim(0, x_max)
        plt.xlabel('km')
        plt.ylabel('km')
        plt.title(map_name)
        plt.savefig('./match_landings/match_{}.png'.format(match_num))
        plt.show()
    else:
        logging.error("Could not get launch data")



def build_drop_data(telemetry_files):  # TODO: Separate by map and patch version
    data_dir = "./data/"
    drop_data = []

    # Plots each match landing locations on a new plot
    for match_num in range(0, len(telemetry_files)):
        logging.debug("Building match %i of %i", match_num, len(telemetry_files) - 1)
        # If the filesize is greater than 0, ie there is actual data in it
        if os.path.getsize(data_dir + telemetry_files[match_num]) > 0:
            try:
                telemetry = load_pickle(data_dir + telemetry_files[match_num])
            except EOFError:
                logging.error("Match file terminated unexpectedly, skipping")
                continue  # Skip processing files that terminate early
            first, last = get_flight_data(telemetry)
            map_name = get_map(telemetry)

            # Set map_size (in cm, like player locations)
            if map_name == "Savage_Main":  # 4km map
                map_size = 400000
            elif map_name == "Erangel_Main":  # 8km map
                map_size = 800000
            elif map_name == "Desert_Main":  # 8km map
                map_size = 800000
            elif map_name == "DihorOtok_Main":  # 6km map
                map_size = 600000

            # Get the flight direction
            if first is not None:
                flight_vec = get_flight_vector(first, last)
                dir = get_flight_category(flight_vec)

            # Get the landings
            landing_locs = get_all_landings(telemetry)
            rankings = get_rankings(telemetry)

            for player, loc in landing_locs.items():
                # For each player who dropped, get their rank from the rankings array
                loc_category = get_loc_cat(loc[0], loc[1], map_size)
                for ranking in rankings:
                    if player in ranking.values():
                        player_rank = ranking['ranking']

                drop_data.append({'player': player,
                                  'drop_loc_raw': loc,
                                  'drop_loc_cat': loc_category,
                                  'rank': player_rank,
                                  'flight_path': dir,
                                  'map': map_name})
                #print("{} ==> {}".format(flight_vec, dir))
            #display_drop_locations(telemetry, plt.figure(), 1, 1, 1, match_num)

    return pd.DataFrame(drop_data)

# note that I am assuming the target is in the last position of the dataframe
# additionally, I am assuming that the list has already been filtered(ie. we are only training on the top players)
# additionally, my current assumption is the data has already been transformed into non-categorical data
def train_model(df, max_k):
    X = df.iloc[:, :-1].copy()#assuming our target is at the very end of the dataframe
    y = df.iloc[:, -1].copy()
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= .2)
    print(x_train)
    print(y_train)
    k_scores = []
    for k in range(1,max_k):
        knn = KNeighborsClassifier(n_neighbors=k)
        cv_scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
        k_scores.append(cv_scores.mean())

    max_score = max(k_scores)

#     get the arg max of the score array to determine the optimnal value for k
    k_opt = None
    for i in range(len(k_scores)):
        if k_scores[i] == max_score:
            k_opt = i + 1

    knn = KNeighborsClassifier(n_neighbors=k_opt)
    kf = KFold(n_splits=10)

   # split the training data into different validation folds and run a training loop on it

    kf.get_n_splits(x_train)
    print(x_train)
    print(y_train)
    for train, test in kf.split(X):
        knn.fit(x_train.loc[train], y_train[train])
        predictions = knn.predict(x_train.loc[test])
        correct_predictions = predictions[predictions == y_train[test]]
        print("Cross val score: ", len(correct_predictions)/ len(predictions))
    training_scores = []
        
        
#     knn.fit(x_train, y_train)#we get about -2.something places per map accuracy
    testing_accuracy = (knn.predict(x_test) - y_test).mean()
    print("TESTING ACCURACY: ", testing_accuracy)
    return knn



# preprocess the dataframe
def preprocess_data(drop_data):
    drop_data = drop_data.drop(columns = ['player'])#probably don't need to include the player in the model
    drop_data = drop_data.drop(columns = ['drop_loc_raw'])#probably don't need to include the player in the model
    drop_data = drop_data.dropna()
    drop_data = drop_data[drop_data['rank'] <= 5]
    labelencoder_X=LabelEncoder()
    X = drop_data.iloc[:,:].values
    drop_data['flight_path'] = labelencoder_X.fit_transform(X[:, 1])
    drop_data['map'] = labelencoder_X.fit_transform(X[:, 2])
    drop_data = drop_data.drop(columns = ['rank'])
    drop_data = drop_data[['flight_path', 'map', 'drop_loc_cat']]
    scaler = MinMaxScaler()
    drop_data.loc[:,:-1] = scaler.fit_transform(drop_data[drop_data.columns[:-1]])
    print("PRINTING DROP DATA!!!!!!")
    print(drop_data)
    return drop_data




# Get safe zone and poison zone states (location and radius) throughout the game
# Returns list of dictionaries representing each time & states
# [ { '_D': str,
#     'safetyZonePosition' : {location},
#     'safetyZoneRadius' : int, 
#       ... }, ... ]
def getZoneStates(json_object):
    logGameStates = search(json_object, None, None, None, 'LogGameStatePeriodic')
    allStates = []
    for gameState in logGameStates:
        timestamp = gameState['_D']
        state = gameState['gameState']
        newStateObj = {k : state[k] for k in ('safetyZonePosition', 
                                                    'safetyZoneRadius', 
                                                    'poisonGasWarningPosition',
                                                    'poisonGasWarningRadius')}
        newStateObj['_D'] = timestamp
        allStates.append(newStateObj)
    return allStates
def get_drop_data():
    data_dir = "./data/"
    match_files = []
    telemetry_files = []

    downloader.setup_logging()
    logging.info("Scanning for match and telemetry files in %s to parse", data_dir)
    for file in os.listdir(data_dir):
        if "_match" in file:
            logging.debug("Match file %s found, adding as match", file)
            match_files.append(file)
        elif "_telemetry" in file:
            logging.debug("Telemetry file %s found, adding as match", file)
            telemetry_files.append(file)

    drop_data = build_drop_data(telemetry_files)
    return drop_data
# def main():
#     drop_data = get_drop_data()
#     map_savage_data = preprocess_data(drop_data[drop_data['map'] == "Savage_Main"])
#     map_erangel_data = preprocess_data(drop_data[drop_data["map"] == "Erangel_Main"])
#     map_desert_data = preprocess_data(drop_data[drop_data['map'] == 'Desert_Main'])
#     max_k = 20# training model hyperparam, anything above this doesn't tell us much
    
#     print("######PRINTING RESULTS FOR DROP LOCATION PREDICTIONS##########\n\n")
#     print("PRINTING SAVAGE_MAIN RESULTS: ")
#     train_model(map_savage_data, max_k)
#     print("PRINTING ERANGEL_MAIN RESULTS: ")
#     train_model(map_erangel_data, max_k)
#     print("PRINTING DESERT_MAIN RESULTS: ")
#     train_model(map_desert_data, max_k)
    
#     print("###########DONE PRINTING DROP LOCATIONS PREDICTIONS###########\n\n")


# if __name__ == "__main__":
#     main()

In [2]:
drop_data = get_drop_data()
drop_data

INFO:root:Scanning for match and telemetry files in ./data/ to parse


2019-06-01 10:26:15,413 - root - INFO - Scanning for match and telemetry files in ./data/ to parse


DEBUG:root:Match file d30210f2-91af-4f89-9aa5-3642149b43bb_match.pickle found, adding as match


2019-06-01 10:26:15,422 - root - DEBUG - Match file d30210f2-91af-4f89-9aa5-3642149b43bb_match.pickle found, adding as match


DEBUG:root:Telemetry file 99f45d22-6e8a-4c32-9d7f-a19cb7eba2c5_telemetry.pickle found, adding as match


...

DEBUG:root:Building match 1915 of 1916


2019-06-01 10:37:21,753 - root - DEBUG - Building match 1915 of 1916


DEBUG:root:Building match 1916 of 1916


2019-06-01 10:37:22,188 - root - DEBUG - Building match 1916 of 1916


,drop_loc_cat,drop_loc_raw,flight_path,map,player,rank
0,QH,"[339081.09375, 152563.390625]",nw,Savage_Main,Hisen28,3
1,MG,"[255137.5625, 135309.015625]",nw,Savage_Main,4am__saobai,20
2,RI,"[349186.90625, 160740.125]",nw,Savage_Main,Lee-Poet,3
3,RH,"[345526.09375, 154371.609375]",nw,Savage_Main,Cameronww,3
4,SH,"[379695.09375, 149027.328125]",nw,Savage_Main,ToveloXxInJ,2
5,MG,"[257949.140625, 135659.703125]",nw,Savage_Main,SHUGESHIGOU,18
6,SG,"[366499.625, 120417.921875]",nw,Savage_Main,helloyaji,2
7,MG,"[255792.421875, 138566.71875]",nw,Savage_Main,Mikkokevin,25
8,MG,"[247687.578125, 136043.0625]",nw,Savage_Main,banji975,13
9,MH,"[253958.6875, 142981.09375]",nw,Savage_Main,China-J-X,13


In [3]:
drop_data[drop_data['map'] == 'DihorOtok_Main']

,drop_loc_cat,drop_loc_raw,flight_path,map,player,rank
1253,PE,"[461252.875, 121505.6015625]",nw,DihorOtok_Main,ing_a_king92,12
1254,NE,"[390034.625, 143004.609375]",nw,DihorOtok_Main,RAMACH817,25
1255,QE,"[482891.875, 120006.8515625]",nw,DihorOtok_Main,kalkaniii,15
1256,PE,"[476173.1875, 130874.5]",nw,DihorOtok_Main,mj_de1111,27
1257,MF,"[375645.71875, 151135]",nw,DihorOtok_Main,ARUY0A,6
1258,PE,"[466002.8125, 124096.6953125]",nw,DihorOtok_Main,dori1986,27
1259,NE,"[411962.90625, 125416.75]",nw,DihorOtok_Main,Poco_s,6
1260,OG,"[448659.03125, 201072.53125]",nw,DihorOtok_Main,18TY_Godmingtol,1
1261,MF,"[384154.46875, 156552.1875]",nw,DihorOtok_Main,HardCouqueDasse,25
1262,PD,"[462944.625, 116282.015625]",nw,DihorOtok_Main,sandnomemer,12


In [4]:
#drop_data = drop_data.drop(columns = ['drop_loc_raw'])
counts = drop_data.groupby(['drop_loc_cat'])['map', 'flight_path'].size().reset_index(name='counts')
counts
drop_data = pd.merge(drop_data, counts, on= 'drop_loc_cat')
temp = drop_data[drop_data['counts'] > 20]
temp

,drop_loc_cat,drop_loc_raw,flight_path,map,player,rank,counts
0,QH,"[339081.09375, 152563.390625]",nw,Savage_Main,Hisen28,3,1235
1,QH,"[337454.65625, 153223.765625]",nn,Savage_Main,XXzqXX,40,1235
2,QH,"[666446.9375, 307975.59375]",ss,Erangel_Main,UNSA_DOTS_long,16,1235
3,QH,"[338379.25, 153301.21875]",sw,Savage_Main,fakeyoumamu,16,1235
4,QH,"[337053.125, 155554.5625]",sw,Savage_Main,CcFei-F,14,1235
5,QH,"[658121.0625, 313172.3125]",ww,Desert_Main,KSTV-BOSS-M,4,1235
6,QH,"[668221.8125, 310822.375]",ww,Desert_Main,18cm_WENDU,25,1235
7,QH,"[676232.4375, 295526.40625]",ww,Desert_Main,yuwananan,22,1235
8,QH,"[664650.0625, 281467.78125]",ww,Desert_Main,jump520,24,1235
9,QH,"[339754.25, 152431.40625]",ww,Savage_Main,FBI-TV-QYTy,13,1235


In [5]:
temp = temp.drop(columns = ['counts'])
drop_data =temp
drop_data

,drop_loc_cat,drop_loc_raw,flight_path,map,player,rank
0,QH,"[339081.09375, 152563.390625]",nw,Savage_Main,Hisen28,3
1,QH,"[337454.65625, 153223.765625]",nn,Savage_Main,XXzqXX,40
2,QH,"[666446.9375, 307975.59375]",ss,Erangel_Main,UNSA_DOTS_long,16
3,QH,"[338379.25, 153301.21875]",sw,Savage_Main,fakeyoumamu,16
4,QH,"[337053.125, 155554.5625]",sw,Savage_Main,CcFei-F,14
5,QH,"[658121.0625, 313172.3125]",ww,Desert_Main,KSTV-BOSS-M,4
6,QH,"[668221.8125, 310822.375]",ww,Desert_Main,18cm_WENDU,25
7,QH,"[676232.4375, 295526.40625]",ww,Desert_Main,yuwananan,22
8,QH,"[664650.0625, 281467.78125]",ww,Desert_Main,jump520,24
9,QH,"[339754.25, 152431.40625]",ww,Savage_Main,FBI-TV-QYTy,13


In [6]:
map_savage_data = drop_data[drop_data['map'] == "Savage_Main"]
counts = map_savage_data.groupby(['drop_loc_cat'])['map', 'flight_path'].size().reset_index(name='counts')
# map_savage_data = preprocess_data(drop_data[drop_data['map'] == "Erangel_Main"])
map_savage_data.index = range(len(map_savage_data.index))
# print(map_savage_data)
# counts = map_savage_data.groupby(['drop_loc_cat'])['map', 'flight_path'].size().reset_index(name='counts')
# counts
map_savage_data = pd.merge(map_savage_data, counts, on= 'drop_loc_cat')
# temp = drop_data[drop_data['counts'] > 20]
# temp
map_savage_data = map_savage_data[map_savage_data['counts'] > 20]
map_savage_data
map_savage_data.drop(columns = ['counts'])
map_savage_data = preprocess_data(map_savage_data)
train_model(map_savage_data, 20)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


PRINTING DROP DATA!!!!!!
       flight_path  map drop_loc_cat
0         0.428571  0.0           QH
8         0.285714  0.0           QH
9         0.000000  0.0           QH
22        1.000000  0.0           QH
38        0.571429  0.0           QH
43        0.428571  0.0           QH
44        0.428571  0.0           QH
45        0.000000  0.0           QH
46        0.000000  0.0           QH
66        0.142857  0.0           QH
83        1.000000  0.0           QH
89        0.142857  0.0           QH
100       0.571429  0.0           QH
101       0.428571  0.0           QH
105       0.428571  0.0           QH
106       0.428571  0.0           QH
116       0.285714  0.0           QH
117       0.285714  0.0           QH
119       1.000000  0.0           QH
120       1.000000  0.0           QH
123       0.142857  0.0           QH
134       0.285714  0.0           QH
137       0.142857  0.0           QH
149       0.142857  0.0           QH
150       0.571429  0.0           QH
166       0.5

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/pyth

       flight_path  map
48952     0.714286  0.0
66032     0.142857  0.0
45322     1.000000  0.0
15532     0.428571  0.0
70790     0.285714  0.0
54885     0.857143  0.0
64565     0.142857  0.0
45310     0.000000  0.0
70250     0.000000  0.0
2548      0.285714  0.0
60145     0.000000  0.0
75305     0.000000  0.0
51013     0.857143  0.0
72269     0.857143  0.0
65506     0.142857  0.0
35817     0.142857  0.0
48341     1.000000  0.0
34377     0.285714  0.0
65156     0.285714  0.0
3047      0.428571  0.0
57779     1.000000  0.0
80768     1.000000  0.0
47747     0.000000  0.0
48675     0.285714  0.0
78808     1.000000  0.0
73560     0.142857  0.0
31468     0.714286  0.0
37800     0.285714  0.0
19808     0.857143  0.0
38814     0.428571  0.0
...            ...  ...
70395     0.142857  0.0
60050     1.000000  0.0
7114      0.428571  0.0
53772     0.000000  0.0
37772     0.142857  0.0
60810     1.000000  0.0
68303     0.142857  0.0
71485     0.857143  0.0
53675     0.857143  0.0
10652     0.4285

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:387: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/series.py:942: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
drop_data = drop_data.dropna()
train_model(drop_data, 20)

In [ ]:
a = np.array(['a', 'b', 'f', 'd'])
b = np.array(['f', 'b', 'e', 'd'])
a[a == b]